In [ ]:
!pip install dalex

In [3]:
#@title

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dalex as dx
from copy import deepcopy, copy
from sklearn.model_selection import train_test_split
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv')

In [6]:
#@title
def encoder_data(df):
  data = df.copy()
  a = pd.get_dummies(data['cp'], prefix = 'cp')
  b = pd.get_dummies(data['thal'], prefix = 'thal')
  c = pd.get_dummies(data['slope'], prefix = 'slope')
  d = pd.get_dummies(data['restecg'], prefix = 'restecg')
  frames = [data.drop(columns = ['cp', 'thal', 'slope', 'restecg']), a, b, c, d]
  data = pd.concat(frames, axis = 1)
  return data

In [7]:
df.ca.replace(4, 0, inplace=True)
df.thal.replace(0, 2, inplace=True)
df = encoder_data(df)
x = df.drop('target', axis=1)
y = df.target

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state=1 , stratify = y)

In [9]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)
print(cross_val_score(lr, x_train, y_train, cv =10, scoring='accuracy'))

[0.84       0.84       0.875      0.875      0.79166667 0.83333333
 0.875      0.83333333 0.91666667 0.91666667]


In [ ]:
#@title
explainer = dx.Explainer(lr, x_train, y_train)

# Analiza pojedyńczego przypaku
Analizujemy pojedyńczego pacjenta: jest to mężczyzna w wieku 45 lat, ze stosunkowo niskim ciśnieniem krwi za to dużym cholesterolem, jest on faktycznie chory.
Główne zmienne dzięki którym diagnozujemy go jako chorego to:
- Naprawiona wada (thal_2 = 1) warto zwórcić uwagę, że pomimo występowania jako trzecia na wykresie to właśnie ta zminna jest kluczowa ponieważ, (thal 3 = 0) wpływa również na plus diagnozy, a jedno wyklucza występowanie drugiej zmiennej 
- brak naczyń zabarwionych w czsie fluoroskopii (ca)
- występownie bólu w klatce (cp_0 = 0) co ciekawe nie istotne jaki ból się pojwaił ale sam fakt występowania.

Zaskakujące może wydawać się, że duży cholesterol nie wpływa na pozywtyne zdiagnowoanie. Podejrzewam, że wynika to z metody Break Down, wydaje się, że zmienne ciągłe nie są dla niej aż tak istotne.

In [11]:
x_train.iloc[3]

age           45.0
sex            1.0
trestbps     128.0
chol         308.0
fbs            0.0
thalach      170.0
exang          0.0
oldpeak        0.0
ca             0.0
cp_0           0.0
cp_1           1.0
cp_2           0.0
cp_3           0.0
thal_1         0.0
thal_2         1.0
thal_3         0.0
slope_0        0.0
slope_1        0.0
slope_2        1.0
restecg_0      1.0
restecg_1      0.0
restecg_2      0.0
Name: 81, dtype: float64

## 1

In [12]:
explainer.predict(x_test)[3]

0.929020023984435

## 2

In [13]:
pp = explainer.predict_parts(x_test.iloc[3])
pp.plot()

In [ ]:
#Kod do znajdowania przypadków do pkt 3
'''
pp1 = explainer.predict_parts(x_test.iloc[11])
ls = []
for i in range(1,x_test.shape[0]):
  pp = explainer.predict_parts(x_test.iloc[i-1])
  if any(pp.result.iloc[:3,0] == 'ca') or any(pp.result.iloc[:3,0] == 'cp_0'):
    continue
  else:
    print(str(i-1))
'''

"\npp1 = explainer.predict_parts(x_test.iloc[11])\nls = []\nfor i in range(1,x_test.shape[0]):\n  pp = explainer.predict_parts(x_test.iloc[i-1])\n  if any(pp.result.iloc[:3,0] == 'ca') or any(pp.result.iloc[:3,0] == 'cp_0'):\n    continue\n  else:\n    print(str(i-1))\n"

# Analiza porównawcza z porzednim przypadkiem
Porównuję do poprzedniej obserwacji.

W przypadu  tego 59 letniego męszczyzny najważniejszymi zmiennymi są:
- obniżenie ST na elektrogramie
- ciśnienie spoczynkowe

Przewidywania te są nieefektywne ponieważ w rzeczywistości pacjent jest chory. 
Ogolnie tak duży spadek ST wskazuje na problemy z sercem przeciwnie do tego co wskazuje mój model, tak samo duże ciśnienie powinno przemawiać za występowaniem choroby. Duży spadek prawdopodobieństwa wystąpienia choroby jest spowodowany istniniem nieodwracalnej zmiany serca. 
Ten przykład wskazuje, że ten model jest źle zaprojektowany. 


## 3

In [14]:
pp = explainer.predict_parts(x_test.iloc[15])
pp.plot()

In [16]:
pp = explainer.predict_parts(x_test.iloc[3])
pp.plot()

# Wpływ zmiennych na wynik predykcji
W tym przypadku próbowałem wybrać dwie osoby dla których ta sama zminna wpływa przeciwnie, ale po przeszukaniu praktycznie wszystkich obserwacji nie udało mi się takiej znaleźć. Jest to dość nieintuicyjne dlatego postanowiłem sprawdzić czy poszczególne zminne. Kodem poniżej sprawdzam, czy istnieje obserwacja która posiada thal_2 = 0 i ma wpływ dodatni na *prediction*. Nie istnieje, w taki sam spobób sprawdziłem wszystkie inne zmienne kategoryczne.

In [17]:
for i in range(1,x_train.shape[0]):
  pp = explainer.predict_parts(x_train.iloc[i-1], B = 1)
  v = pp.result.loc[pp.result.variable_name == 'exang',['variable_value']]
  s = pp.result.loc[pp.result.variable_name == 'exang',['sign']]
  if v.iloc[0,0] == '0.0' and s.iloc[0,0] == -1:
    print(i)
  if v.iloc[0,0] == '1.0'  and s.iloc[0,0] == 1:
    print(i)


In [18]:
pp = explainer.predict_parts(x_test.iloc[10])
pp.plot()

In [19]:
pp = explainer.predict_parts(x_test.iloc[15])
pp.plot()

In [20]:
pp = explainer.predict_parts(x_test.iloc[20])
pp.plot()

In [ ]:
!pip install nbconvert

In [27]:
!jupyter nbconvert --to html HW_Patryk_Slowakiewicz.ipynb

[NbConvertApp] WARNING | pattern u'HW_Patryk_Slowakiewicz.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--inplac